In [1]:
## 🚀 Setup and Helper Functions

import json
from datetime import datetime, timedelta
import glob
import os

# --- Helper function for half-second window calculation ---

def get_half_second_target(ts_datetime):
    """
    Calculates the nearest .000 or .500 target time for a given datetime,
    used for rounding targets and defining 500ms windows.
    """
    milliseconds = ts_datetime.microsecond // 1000
    remainder = milliseconds % 500

    if remainder < 250:
        milliseconds -= remainder
    else:
        milliseconds += (500 - remainder)

    if milliseconds == 1000:
        # Handle overflow to the next second
        target = ts_datetime + timedelta(seconds=1)
        target = target.replace(microsecond=0)
    else:
        target = ts_datetime.replace(microsecond=milliseconds * 1000)

    return target

# --- Helper function for 100ms rounding ---

def round_to_100ms(ts_datetime):
    """
    Rounds a datetime object to the nearest 100 milliseconds (one decimal of a second).
    """
    total_milliseconds = (ts_datetime.microsecond // 1000)

    # Round the milliseconds to the nearest multiple of 100
    rounded_milliseconds = round(total_milliseconds / 100) * 100

    # Handle overflow from 999ms to 1000ms (which is 1 second)
    if rounded_milliseconds == 1000:
        dt_rounded = ts_datetime + timedelta(seconds=1)
        dt_rounded = dt_rounded.replace(microsecond=0)
    else:
        # Update the microsecond part
        dt_rounded = ts_datetime.replace(microsecond=rounded_milliseconds * 1000)

    return dt_rounded


def final_unique_resolution(all_moves_list):
    """
    Performs a final chronological pass to ensure all timestamps are strictly
    increasing, using a 100ms increment for collisions. This is the last step.
    """
    # Sort by the tentatively assigned final_ts_dt
    all_moves_list.sort(key=lambda x: x['final_ts_dt'])

    previous_final_ts = None

    for move in all_moves_list:
        final_ts = move['final_ts_dt']

        # Chronological uniqueness check: push it past the previous move
        if previous_final_ts and final_ts <= previous_final_ts:
            final_ts = previous_final_ts + timedelta(milliseconds=100)

        move['final_ts_dt'] = final_ts
        previous_final_ts = final_ts

    return all_moves_list


def update_timestamps(data):
    """
    Applies the multi-stage, prioritized, and filtered timestamp update process.
    """

    original_game_moves = data['gameMoves']
    original_do_something_timestamps = data['doSomethingTimestamps']

    final_timestamps_set = set() # To track the assigned slots (only for half-second check)

    # --- 1. Process gameMoves (100ms round, then .0/.5 priority) ---
    processed_gm = []
    sorted_gm_items = sorted(original_game_moves.items())

    for ts_str, value in sorted_gm_items:
        original_dt = datetime.fromisoformat(ts_str.replace('Z', '+00:00'))

        # 1a. Round to the nearest 100ms
        dt_100ms = round_to_100ms(original_dt)

        # 1b. Check for closest .0 or .5 target
        target_half_second = get_half_second_target(dt_100ms)

        assigned_ts = dt_100ms # Default to the 100ms rounded time

        # Priority rule: Round to .0 or .5 if available
        if target_half_second not in final_timestamps_set:
            assigned_ts = target_half_second

        processed_gm.append({
            'final_ts_dt': assigned_ts,
            'type': 'gameMoves',
            'value': value
        })
        # Tentative assignment added to the set to block lower priority moves
        final_timestamps_set.add(assigned_ts)

    # --- 2. Process doSomethingTimestamps (100ms round, then 500ms filtering) ---
    filtered_dst = []
    half_second_windows = set() # Tracks the 500ms windows that already have a move
    sorted_dst_items = sorted(original_do_something_timestamps.items())

    for ts_str, value in sorted_dst_items:
        original_dt = datetime.fromisoformat(ts_str.replace('Z', '+00:00'))

        # 2a. Round to the nearest 100ms
        dt_100ms = round_to_100ms(original_dt)

        # 2b. Determine the 500ms window key (e.g., 33:35.000)
        window_dt = get_half_second_target(dt_100ms)

        # Filtering rule: Keep only the first move in this 500ms window
        if window_dt not in half_second_windows:
            filtered_dst.append({
                'final_ts_dt': dt_100ms, # Assigned time is the 100ms rounded value
                'type': 'doSomethingTimestamps',
                'value': value
            })
            half_second_windows.add(window_dt)

    # 3. Combine for final resolution
    all_moves_list = processed_gm + filtered_dst

    # 4. Final Collision Resolution and Timestamp Assignment (Enforce global uniqueness)
    resolved_moves_list = final_unique_resolution(all_moves_list)

    # 5. Final Rebuild of Dictionaries
    new_game_moves = {}
    new_do_something_timestamps = {}

    for move in resolved_moves_list:
        final_ts_str = move['final_ts_dt'].isoformat().replace('+00:00', 'Z')

        if move['type'] == 'gameMoves':
            new_game_moves[final_ts_str] = move['value']
        elif move['type'] == 'doSomethingTimestamps':
            new_do_something_timestamps[final_ts_str] = move['value']

    data['gameMoves'] = new_game_moves
    data['doSomethingTimestamps'] = new_do_something_timestamps

    return data

# --- Main processing function (remains the same) ---

def process_files(file_list, output_prefix='_updated'):
    """
    Processes a list of JSON files to update timestamps and saves the
    result to a new file with the specified prefix.
    """
    print(f"--- Starting file processing for {len(file_list)} files ---")

    for input_filepath in file_list:
        try:
            # 1. Determine the output filename
            directory = os.path.dirname(input_filepath)
            filename_with_ext = os.path.basename(input_filepath)
            name, ext = os.path.splitext(filename_with_ext)

            # Construct the new filename: insert '_updated' before the final number
            last_underscore_index = name.rfind('_')
            if last_underscore_index != -1 and name[last_underscore_index+1:].isdigit():
                base_name = name[:last_underscore_index]
                suffix = name[last_underscore_index:]
                new_name = f"{base_name}{output_prefix}{suffix}"
            else:
                new_name = f"{name}{output_prefix}"

            output_filepath = os.path.join(directory, f"{new_name}{ext}")

            print(f"\nProcessing: {input_filepath}")

            # 2. Load the data
            with open(input_filepath, 'r') as f:
                data = json.load(f)

            # Check original lengths before update
            original_gm_len = len(data['gameMoves'])
            original_dst_len = len(data['doSomethingTimestamps'])
            total_original = original_gm_len + original_dst_len

            # 3. Update the timestamps
            updated_data = update_timestamps(data)

            # Check final lengths after update
            final_gm_len = len(updated_data['gameMoves'])
            final_dst_len = len(updated_data['doSomethingTimestamps'])
            total_final = final_gm_len + final_dst_len

            # Verify that GameMoves count is preserved (DST count is expected to drop due to filtering)
            if original_gm_len != final_gm_len:
                raise ValueError(
                    f"DATA MISMATCH: Original GameMoves count ({original_gm_len}) "
                    f"does not match final GameMoves count ({final_gm_len}). "
                    "DST count reduction is expected but GM count must be preserved."
                )

            # 4. Save the new data
            with open(output_filepath, 'w') as f:
                json.dump(updated_data, f, indent=4)

            print(f"Output to: {output_filepath}")
            print(f"Successfully updated and saved. Total original moves: {total_original}, final moves: {total_final}")

        except Exception as e:
            cleaned_path = input_filepath.replace('\\', '/')
            print(f"!!! FATAL ERROR processing {cleaned_path}: {e}")

# --- Execution Cell ---

# Define the pattern to find all relevant JSON files.
# NOTE: If you run this in a different environment, you may need to adjust the file_pattern
# or the directory structure for files to be found.
file_pattern = '../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)/*.json'

# Get the list of all files to process
all_files_to_process = glob.glob(file_pattern)

# Execute the main function
process_files(all_files_to_process)

--- Starting file processing for 11 files ---

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_1.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_1.json
Successfully updated and saved. Total original moves: 472, final moves: 383

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_10.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_updated_10.json
Successfully updated and saved. Total original moves: 640, final moves: 426

Processing: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)_2.json
Output to: ../output/all/Two, one and one (2 SHORT_FUSE, 1 BASIC, 1 REACTIVE)\Two, one a